In [1]:
import requests

from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq


model = AutoModelForVision2Seq.from_pretrained("ydshieh/kosmos-2-patch14-224", trust_remote_code=True)
processor = AutoProcessor.from_pretrained("ydshieh/kosmos-2-patch14-224", trust_remote_code=True)

# url = "https://huggingface.co/ydshieh/kosmos-2-patch14-224/resolve/main/snowman.png"
# image = Image.open(requests.get(url, stream=True).raw)
path = "/media/data/qiyang/data/OH0037_2023_05_10_15_17_17/frames/Aoutput0_cut/frame_0.jpg"
image = Image.open(path)

def run_example(prompt):

    inputs = processor(text=prompt, images=image, return_tensors="pt")
    generated_ids = model.generate(
        pixel_values=inputs["pixel_values"],
        input_ids=inputs["input_ids"][:, :-1],
        attention_mask=inputs["attention_mask"][:, :-1],
        img_features=None,
        img_attn_mask=inputs["img_attn_mask"][:, :-1],
        use_cache=True,
        max_new_tokens=64,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    _processed_text = processor.post_process_generation(generated_text, cleanup_and_extract=False)
    processed_text, entities = processor.post_process_generation(generated_text)
    print(processed_text)
    print(entities)
    print(_processed_text)


/home/qiyang/anaconda3/envs/kosmos-2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Phase Grounding

In [2]:
prompt = "<grounding><phrase> a man</phrase>"
run_example(prompt)

# a snowman is warming himself by the fire
# [('a snowman', (0, 9), [(0.390625, 0.046875, 0.984375, 0.828125)]), ('the fire', (32, 40), [(0.203125, 0.015625, 0.453125, 0.859375)])]

# <grounding><phrase> a snowman</phrase><object><patch_index_0044><patch_index_0863></object> is warming himself by<phrase> the fire</phrase><object><patch_index_0006><patch_index_0878></object>


a man plays chess on a table with a built-in chessboard
[('a man', (0, 5), [(0.296875, 0.015625, 0.515625, 0.421875)]), ('chess', (12, 17), [(0.453125, 0.203125, 0.765625, 0.515625)]), ('a table', (21, 28), [(0.203125, 0.171875, 0.546875, 0.828125)]), ('a built-in chessboard', (34, 55), [(0.234375, 0.578125, 0.703125, 0.984375)])]
<grounding><phrase> a man</phrase><object><patch_index_0009><patch_index_0432></object> plays<phrase> chess</phrase><object><patch_index_0206><patch_index_0536></object> on<phrase> a table</phrase><object><patch_index_0166><patch_index_0849></object> with<phrase> a built-in chessboard</phrase><object><patch_index_0583><patch_index_1014></object>


### Referring Expression Comprehension 

In [3]:
prompt = "<grounding><phrase> a man next to a table</phrase>"
run_example(prompt)

# a snowman next to a fire
# [('a snowman next to a fire', (0, 24), [(0.390625, 0.046875, 0.984375, 0.828125)])]

# <grounding><phrase> a snowman next to a fire</phrase><object><patch_index_0044><patch_index_0863></object>


a man next to a table plays a game of chess
[('a man next to a table', (0, 21), [(0.296875, 0.015625, 0.515625, 0.421875)]), ('a game of chess', (28, 43), [(0.453125, 0.203125, 0.765625, 0.515625)])]
<grounding><phrase> a man next to a table</phrase><object><patch_index_0009><patch_index_0432></object> plays<phrase> a game of chess</phrase><object><patch_index_0206><patch_index_0536></object>


### Grounded Image captioning 

Brief

In [4]:
prompt = "<grounding> An image of"
run_example(prompt)

# An image of a snowman warming himself by a campfire.
# [('a snowman', (12, 21), [(0.390625, 0.046875, 0.984375, 0.828125)]), ('a campfire', (41, 51), [(0.109375, 0.640625, 0.546875, 0.984375)])]

# <grounding> An image of<phrase> a snowman</phrase><object><patch_index_0044><patch_index_0863></object> warming himself by<phrase> a campfire</phrase><object><patch_index_0643><patch_index_1009></object>.


An image of a man playing a chess game on a table.
[('a man', (12, 17), [(0.296875, 0.015625, 0.515625, 0.421875)]), ('a chess game', (26, 38), [(0.453125, 0.203125, 0.765625, 0.515625)]), ('a table', (42, 49), [(0.203125, 0.171875, 0.546875, 0.828125)])]
<grounding> An image of<phrase> a man</phrase><object><patch_index_0009><patch_index_0432></object> playing<phrase> a chess game</phrase><object><patch_index_0206><patch_index_0536></object> on<phrase> a table</phrase><object><patch_index_0166><patch_index_0849></object>.


Detailed 

In [5]:
prompt = "<grounding> Describe this image in detail:"
run_example(prompt)

# Describe this image in detail: The image features a snowman sitting by a campfire in the snow. He is wearing a hat, scarf, and gloves, with a pot nearby and a cup
# [('a campfire', (71, 81), [(0.171875, 0.015625, 0.484375, 0.984375)]), ('a hat', (109, 114), [(0.515625, 0.046875, 0.828125, 0.234375)]), ('scarf', (116, 121), [(0.515625, 0.234375, 0.890625, 0.578125)]), ('gloves', (127, 133), [(0.515625, 0.390625, 0.640625, 0.515625)]), ('a pot', (140, 145), [(0.078125, 0.609375, 0.265625, 0.859375)])]

# <grounding> Describe this image in detail: The image features a snowman sitting by<phrase> a campfire</phrase><object><patch_index_0005><patch_index_1007></object> in the snow. He is wearing<phrase> a hat</phrase><object><patch_index_0048><patch_index_0250></object>,<phrase> scarf</phrase><object><patch_index_0240><patch_index_0604></object>, and<phrase> gloves</phrase><object><patch_index_0400><patch_index_0532></object>, with<phrase> a pot</phrase><object><patch_index_0610><patch_index_0872></object> nearby and<phrase> a cup</phrase><object>


Describe this image in detail: A man is standing in a kitchen, holding a chess board. He is looking down at a white and black chessboard on a table.
[('A man', (31, 36), [(0.296875, 0.015625, 0.515625, 0.421875)]), ('a chess board', (71, 84), [(0.453125, 0.203125, 0.765625, 0.515625)]), ('a white and black chessboard on a table', (108, 147), [(0.234375, 0.578125, 0.703125, 0.984375)])]
<grounding> Describe this image in detail:<phrase> A man</phrase><object><patch_index_0009><patch_index_0432></object> is standing in a kitchen, holding<phrase> a chess board</phrase><object><patch_index_0206><patch_index_0536></object>. He is looking down at<phrase> a white and black chessboard on a table</phrase><object><patch_index_0583><patch_index_1014></object>.
